# Modular launch notebook.

This notebook can be used as an alternative to launch.py.



<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/modular/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup and imports

In [ ]:
!pip install -r https://raw.githubusercontent.com/google-deepmind/concordia/main/examples/requirements.txt
!pip install --ignore-requires-python git+https://github.com/google-deepmind/concordia.git

#@markdown Here we check if this is being ran in colab or codespaces. If running a colab, we clone the repository to get the example environments
import os
import sys
import pathlib

if os.environ.get('CODESPACES') == 'true':
  print("Running in Codespaces")
  concordia_root_dir = pathlib.Path.cwd().parent.parent.resolve()
  sys.path.append(f'{concordia_root_dir}')
elif 'google.colab' in sys.modules:
  print("Running in Colab")
  !git clone https://github.com/google-deepmind/concordia.git
  sys.path.append('/content/concordia/')
else:
  print("Running in a different environment, please make sure to add root directory of examples to with sys.path.append")

In [ ]:
import datetime
import importlib
import os
import pathlib
import sys

from concordia.language_model import amazon_bedrock_model
from concordia.language_model import google_aistudio_model
from concordia.language_model import gpt_model
from concordia.language_model import langchain_ollama_model
from concordia.language_model import mistral_model
from concordia.language_model import no_language_model
from concordia.language_model import ollama_model
from concordia.language_model import pytorch_gemma_model
from concordia.utils import measurements as measurements_lib
import sentence_transformers

## Parameters

In [ ]:
# @title Parameters (edit this cell)

# Pick AGENT_NAME from the factories implemented in concordia/factory/agent.
AGENT_NAME = 'rational_entity_agent__main_role'
# Pick ENVIRONMENT_NAME from the factories in concordia/factory/environment.
ENVIRONMENT_NAME = 'forbidden_fruit'
# Pick from {openai, mistral}. Feel free to implement more!
API_TYPE = 'mistral'
# Pick  a specific model e.g. gpt-4o if openai or codestral-latest if mistral.
# You can select any model listed at https://platform.openai.com/docs/models
# if API_TYPE is openai or https://docs.mistral.ai/getting-started/models/ when
# API_TYPE is mistral. Feel free to implement more!
MODEL_NAME = 'codestral-latest'
# Select an embedder by specifying one of the sentence transformer embedding
# models listed at https://huggingface.co/sentence-transformers.
EMBEDDER_NAME = 'all-mpnet-base-v2'
# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True.
DISABLE_LANGUAGE_MODEL = False

## Load the agent config with importlib

In [ ]:
# @title Load the agent config with importlib

IMPORT_AGENT_BASE_DIR = 'concordia.factory.agent'
agent_module = importlib.import_module(
    f'{IMPORT_AGENT_BASE_DIR}.{AGENT_NAME}')
# Load the environment config with importlib
IMPORT_ENV_BASE_DIR = 'examples.modular.environment'
simulation = importlib.import_module(
    f'{IMPORT_ENV_BASE_DIR}.{ENVIRONMENT_NAME}')

## Language Model setup

In [ ]:
# @title Language Model setup

if DISABLE_LANGUAGE_MODEL:
  model = no_language_model.NoLanguageModel()
elif API_TYPE == 'amazon_bedrock':
  model = amazon_bedrock_model.AmazonBedrockLanguageModel(MODEL_NAME)
elif API_TYPE == 'google_aistudio_model':
  model = google_aistudio_model.GoogleAIStudioLanguageModel(MODEL_NAME)
elif API_TYPE == 'langchain_ollama':
  model = langchain_ollama_model.LangchainOllamaLanguageModel(MODEL_NAME)
elif API_TYPE == 'mistral':
  model = mistral_model.MistralLanguageModel(MODEL_NAME)
elif API_TYPE == 'ollama':
  model = ollama_model.OllamaLanguageModel(MODEL_NAME)
elif API_TYPE == 'openai':
  model = gpt_model.GptLanguageModel(MODEL_NAME)
elif API_TYPE == 'pytorch_gemma':
  model = pytorch_gemma_model.PyTorchGemmaLanguageModel(MODEL_NAME)
else:
  raise ValueError(f'Unrecognized api type: {API_TYPE}')

## Setup sentence encoder

In [ ]:
# @title Setup sentence encoder
_embedder_model = sentence_transformers.SentenceTransformer(
    f'sentence-transformers/{EMBEDDER_NAME}')
embedder = lambda x: _embedder_model.encode(x, show_progress_bar=False)

# The simulation

## Initialize the simulation

In [ ]:
# @title Initialize the simulation
measurements = measurements_lib.Measurements()
runnable_simulation = simulation.Simulation(
    model=model,
    embedder=embedder,
    measurements=measurements,
    agent_module=agent_module,
)

## Run the simulation

In [ ]:
# @title Run the simulation
results_log = runnable_simulation()

## Save the results log

In [ ]:
# @title Write the results log as an HTML file in the current working directory.
filename = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '.html'
file_handle = open(filename, 'a')
file_handle.write(results_log)
file_handle.close()

```
Copyright 2023 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```